In [22]:
import datetime
import pandas_datareader as web
import matplotlib.pyplot as plt

PERIOD = 360
DATE = datetime.date.today()
START = datetime.datetime.now() - datetime.timedelta(days=PERIOD)

sid = '2330'


In [23]:
def analyis(sid):
    df = web.DataReader(sid +'.TW', data_source = 'yahoo', start = START, end = DATE)
    numerator = df['Close']-df['Low'].rolling(window=9).min()
    denominator = df['High'].rolling(window=9).max() - df['Low'].rolling(window=9).min()

    rsv = numerator * 100 / denominator
    rsv.head(10)
    k = []
    d = []
    for i in range (8):
        k.append(50)
        d.append(50)
    
    ##  K = 2/3 X (昨日K值) + 1/3 X (今日RSV)   
    for i in range (8, df.shape[0]):
        a = ((k[i - 1] * 2) + rsv[i]) / 3
        k.append(a)
    
    ## D = 2/3 X (昨日D值) + 1/3 X (今日K值)
    for i in range (8, df.shape[0]):
        b = ((d[i - 1] * 2) + k[i]) / 3
        d.append(b)
    
    df['K'] = k 
    df['D'] = d
    print(df['K'])
    print(df['D'])
    KD_Low = [];
    KD_High = [];
    for i in range (0, df.shape[0]):
        if df['K'][i] < 21 or df['D'][i] < 21:
            KD_Low.append(True)
        else:
            KD_Low.append(False)
        if df['K'][i] >= 80 or df['D'][i] >= 80:
            KD_High.append(True)
        else:
            KD_High.append(False)
    ## volume
    rich = []
    rich.append(False)
    for i in range (1, df.shape[0]):
        if df['Volume'][i] > df['Volume'][i - 1]:
            rich.append(True)
        else:
            rich.append(False) 
        
    print(df['Volume'])
    cBuy = []
    cSell = []
    for i in range (0, df.shape[0]):
        if KD_Low[i] and rich[i]:
            cBuy.append(True)
        else:
            cBuy.append(False)
    for i in range (0, df.shape[0]):
        if KD_High[i] and rich[i] == False:
            cSell.append(True)
        else:
            cSell.append(False)        
    df['Buy'] = cBuy
    df['Sell'] = cSell
   
    avg_price = 0
    number = 0
    profit = 0
    cost = 0
    maxcost = 0
    for i in range (0, df.shape[0]):
        if df['Buy'][i] and number == 0:
            cost = df['Close'][i]
            avg_price = df['Close'][i]
            number = number + 1
            print('average price:',avg_price) 
            print('storage:',number)   
        if df['Buy'][i] and number != 0:
            cost = cost + df['Close'][i]
            avg_price = (avg_price + df['Close'][i]) / (2)
            number = number + 1
            print('average price:',avg_price)    
            print('storage:',number)  
        if KD_High[i] and rich[i] == False and number != 0 and avg_price < df['Close'][i]:
            profit = profit + (df['Close'][i] - avg_price) * number
            if(cost > maxcost):
                maxcost = cost
            cost = 0;
            avg_price = 0
            number = 0
            print('Sell at :', df['Close'][i])

    profit = profit + ((avg_price - df['Close'][df.shape[0] - 1) * number)
    print('Total cost:                ', maxcost * 1000)
    print('Total profit of ' + str(PERIOD) + ' days: %.2f' %(profit * 1000))    
    print('Total Rate of Return:\t\t%.2f' %(profit* 100 / maxcost))
    print(df)  
    

In [24]:
analyis('2303')

Date
2021-01-18    50.000000
2021-01-19    50.000000
2021-01-20    50.000000
2021-01-21    50.000000
2021-01-22    50.000000
                ...    
2022-01-04    46.815766
2022-01-05    35.972390
2022-01-06    27.207393
2022-01-07    20.288768
2022-01-10    16.228541
Name: K, Length: 239, dtype: float64
Date
2021-01-18    50.000000
2021-01-19    50.000000
2021-01-20    50.000000
2021-01-21    50.000000
2021-01-22    50.000000
                ...    
2022-01-04    62.371792
2022-01-05    53.571992
2022-01-06    44.783792
2022-01-07    36.618784
2022-01-10    29.822036
Name: D, Length: 239, dtype: float64
Date
2021-01-18    190288745.0
2021-01-19    549567983.0
2021-01-20    519469908.0
2021-01-21    348371621.0
2021-01-22    421387831.0
                 ...     
2022-01-04    113554504.0
2022-01-05     83002873.0
2022-01-06     80543215.0
2022-01-07     99195601.0
2022-01-10     95558061.0
Name: Volume, Length: 239, dtype: float64
average price: 49.04999923706055
storage: 1
average pri